In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import datetime
import json
import numpy as np
import os
import pandas as pd
import sys 
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import random 

from keras.callbacks import History 
history = History()

BATCH_SIZE = 10
number_of_attributes = 35

number_of_permutations = 33 # for each attribute

print(tf.__version__)

2.8.0


In [ ]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'), 
    tf.keras.layers.Dense(12)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [ ]:
# Select a sub-dataframe from the main dataframe with a lower number of attributes
df = df_raw[attributes_array[0:number_of_attributes]]
df_test = df_test_raw[attributes_array[0:number_of_attributes]]

print(df_test)

# Combine the data with possible classes
numeric_feature_names = attributes_array[0:number_of_attributes]
numeric_features = df[numeric_feature_names] # Data for training
numeric_features_test = df_test[numeric_feature_names] # Data for testing

# Converts pandas dataframe to tensorflow object
numeric_features = tf.convert_to_tensor(numeric_features)
#numeric_features_test = tf.convert_to_tensor(numeric_features_test) botei na célula abaixo

# Normalize the data
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)
#normalizer.adapt(numeric_features_test)

# Execute Trainning and Testing
model = get_basic_model()
print('Starting training for',number_of_attributes,'attributes')
history = model.fit(numeric_features, target, epochs=10, verbose=0, batch_size=BATCH_SIZE, callbacks=[history])
train_accuracy = history.history['accuracy']
print('train_accuracy:', train_accuracy)

       10    15  17    22  24  29  31  32  33  34  ...    81    82   83   84  \
0      66   503  52   489  14  14   7   5   6   5  ...   437   398  282  398   
1      66   465  52   451  14  14   6   5   5   5  ...   282   399  263  399   
2      66   464  52   450  14  14   6   6   5   6  ...   353   398  282  398   
3      66   465  52   451  14  14   6   6   5   6  ...   282   399  272  399   
4      66   682  52   668  14  14   6   5   5   5  ...   616   399  282  399   
...    ..   ...  ..   ...  ..  ..  ..  ..  ..  ..  ...   ...   ...  ...  ...   
23796  60  1514  40  1500  14  20  14  12  13  12  ...  1460    97    6   21   
23797  60  1414  40  1400  14  20  16  13  15  13  ...  1360    97    6   21   
23798  60  1514  40  1500  14  20  14  20  13  20  ...   391  1460  391  759   
23799  60   989  40   975  14  20   7   5   6   5  ...   495   935  456  449   
23800  60  1514  40  1500  14  20  24  29  23  29  ...   489  1460  438    2   

          87     88     89     90  91  

In [ ]:
test_loss, test_acc =  model.evaluate(numeric_features_test, target_test, verbose=0, batch_size=BATCH_SIZE)

permutated_accuracy_means_array = []

#for every attribute
for attr_i, attr in enumerate(numeric_features_test.columns):
  print(f"Running attr: {attr}")  
  
  permutated_accuracy_array = []  
  # run n times for each attribute
  for n in range(number_of_permutations):  
    df_permutated = numeric_features_test.copy()
    df_permutated[attr] = np.random.permutation(df_permutated[attr].values)
    #display('original:',numeric_features_test[attr])
    #display('permutaded',df_permutated[attr])

    df_permutated = tf.convert_to_tensor(df_permutated)      
    permutated_loss, permutated_acc = model.evaluate(df_permutated,  target_test, verbose=0, batch_size=BATCH_SIZE)  
    permutated_accuracy_array.append(permutated_acc)

  print('permutated_accuracy_array:',permutated_accuracy_array)
  permutated_accuracy_mean = sum(permutated_accuracy_array) / len(permutated_accuracy_array)  
  print('permutated_accuracy_mean',permutated_accuracy_mean)
  ratio = permutated_accuracy_mean / test_acc
  print(f"Original Accuracy: {test_acc:.3f} Permutated Accuracy: {permutated_accuracy_mean:.3f} Ratio: {ratio:.3f}\n")

  permutated_accuracy_means_array.append((permutated_accuracy_mean, numeric_feature_names[attr_i])) 
  #print('permutated_accuracy_means_array',permutated_accuracy_means_array,'\n')
  #if attr_i > 3: break

permutated_accuracy_means_array = sorted(permutated_accuracy_means_array)
print('permutated_accuracy_means_array SORTED',permutated_accuracy_means_array)
attributes = [x for _, x in permutated_accuracy_means_array]
print('sorted attributes', attributes)

# Date for report file
current_date_and_time = datetime.date.today()
current_date_and_time_string = str(current_date_and_time)
# Report file content
attr_report = {
    "datetime": current_date_and_time_string,
    "sorted_attributes_by_weights": attributes
}
# Writes into the file
attr_title = "/content/drive/MyDrive/network-traffic-classification-main/nn-attribute-arrays/%s"%current_date_and_time_string
with open(attr_title, "w") as f:
  json.dump(attr_report , f)
with open("/content/drive/MyDrive/network-traffic-classification-main/nn-attribute-arrays/last-run", "w") as f:
  json.dump(attr_report , f)
with open(attr_title, "r") as f:
  rep = json.load(f)
  display(rep)  


Running attr: 10
permutated_accuracy_array: [0.9434897899627686, 0.9423553347587585, 0.941641092300415, 0.9430276155471802, 0.9418091773986816, 0.9428175091743469, 0.9425654411315918, 0.9426494836807251, 0.9422293305397034, 0.9427754878997803, 0.943195641040802, 0.9424393773078918, 0.9422293305397034, 0.9424813985824585, 0.9429015517234802, 0.9427335262298584, 0.9431536197662354, 0.9421452879905701, 0.9425234198570251, 0.9420192241668701, 0.9428175091743469, 0.9423553347587585, 0.9425234198570251, 0.9426074624061584, 0.9429855942726135, 0.9428595304489136, 0.9423973560333252, 0.9421032667160034, 0.9418932199478149, 0.9426494836807251, 0.9416831135749817, 0.9425654411315918, 0.9424813985824585]
permutated_accuracy_mean 0.9425183263691989
Original Accuracy: 0.948 Permutated Accuracy: 0.943 Ratio: 0.994

Running attr: 15
permutated_accuracy_array: [0.9363471865653992, 0.935716986656189, 0.9363471865653992, 0.9369354248046875, 0.9368093609809875, 0.9364312291145325, 0.9358430504798889, 0.9

{'datetime': '2022-04-13',
 'sorted_attributes_by_weights': ['83',
  '82',
  '79',
  '90',
  '88',
  '84',
  '15',
  '17',
  '29',
  '22',
  '89',
  '59',
  '87',
  '10',
  '80',
  '60',
  '64',
  '38',
  '45',
  '33',
  '63',
  '61',
  '36',
  '24',
  '47',
  '31',
  '37',
  '81',
  '62',
  '91',
  '32',
  '46',
  '34',
  '48',
  '92']}